<a href="https://colab.research.google.com/github/QuiD-10/googletest/blob/main/CS_105_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 105 Final Project

This is the Jupyter notebook that contains all the code snippets and explanations used in our project.

We are Group 4: Wesley Hur, Avia Bequette, Benson Lee, Gavin Duong, Megan Yip, Richard Kim

Our project is...

WH 12/4/24 note: I suggest using a separate Colab notebook for testing and then putting finished code snippets here since we could potentially overwrite each other's work. You'll know when this is an issue when there's a red warning where "All changes saved" should be

Useful links for PRAW on Colab:


*   https://www.reddit.com/r/redditdev/comments/bd863h/praw_on_colab/
*  https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab


https://www.reddit.com/r/redditdev/comments/hsn2km/praw_now_has_asynchronous_support_with_async_praw/

Potential subreddits to analyze:


*   https://www.reddit.com/r/PoliticalCompassMemes/

Things to consider:


*   https://michiganross.umich.edu/news/new-study-reddit-explores-how-political-bias-content-moderation-feeds-echo-chambers

*  https://pmc.ncbi.nlm.nih.gov/articles/PMC8907875/
"Ideological variation in preferred content and source credibility on Reddit during the COVID-19 pandemic"


## Scraping our dataset from Reddit

We use Async PRAW to interact with Reddit's API in order to scrape data from Reddit.
Async PRAW is an asynchronous version of PRAW.

In [1]:
# Install Async PRAW to Colab instance
!pip install --upgrade asyncpraw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 3.4 MB/s eta 0:00:00


In [2]:
import asyncpraw
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import XLNetTokenizer, XLNetModel

In [3]:
# Import Google Drive directory (for saving dataset to Drive folder)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Initialize Reddit instance
# Hard-coded API throwaway account, can read public Reddit data

reddit = asyncpraw.Reddit(
    client_id="wD6FvCqLN-SFV8Z16Z32og",
    client_secret="60lYz8F0GGVUf2DBXHHYBuq2UPqUpQ",
    user_agent="U.S. Election 2024 Reddit Scraper (by /u/DesignerSecurity2239)",
)

In [5]:
# Test to see if Async PRAW works
# assume you have a asyncpraw.Reddit instance bound to variable `reddit`
subreddit = await reddit.subreddit("redditdev", fetch=True)

print(subreddit.display_name)
# Output: redditdev
print(subreddit.title)
# Output: reddit development
print(subreddit.description)
# Output: a subreddit for discussion of ...

redditdev
reddit Development
A subreddit for discussion of Reddit's API and Reddit API clients.

* [Read the API Overview & Rules](https://github.com/reddit/reddit/wiki/API)
* [Check out the API documentation](http://www.reddit.com/dev/api)
* [PRAW chat](https://join.slack.com/t/praw/shared_invite/enQtOTUwMDcxOTQ0NzY5LWVkMGQ3ZDk5YmQ5MDEwYTZmMmJkMTJkNjBkNTY3OTU0Y2E2NGRlY2ZhZTAzMWZmMWRiMTMwYjdjODkxOGYyZjY)
* [Snoowrap chat](https://gitter.im/not-an-aardvark/snoowrap)
* [Unofficial Discord](https://discord.gg/hVMhxpV)
* Please do not request bots here. Consider /r/requestabot instead.


Please confine discussion to Reddit's API instead of using this as a soapbox to talk to the admins. In particular, use [/r/ideasfortheadmins](/r/ideasfortheadmins) for feature ideas and [/r/bugs](/r/bugs) for bugs. If you have general reddit questions, try [/r/help](/r/help).

To see an explanation of recent user-facing changes to reddit (and the code behind them), check out /r/changelog.

---

To report a

In [6]:
subreddit = await reddit.subreddit("politics", fetch=True)
# assume you have a Subreddit instance bound to variable `subreddit`
async for submission in subreddit.hot(limit=1):
    print(submission.title)
    # Output: the submission's title
    print(submission.score)
    # Output: the submission's score
    print(submission.id)
    # Output: the submission's ID
    print(submission.url)
    # Output: the URL the submission points to or the submission's URL if it's a self post

r/Politics 2024 US Elections Prediction Contest Results!
21
1h84irc
https://www.reddit.com/r/politics/comments/1h84irc/rpolitics_2024_us_elections_prediction_contest/


In [7]:
# Use two separate DataFrames for posts and comments

## NLTK processing

In [8]:
# NLTK stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words("english"))
text = submission.title + " " + submission.selftext
tokens = word_tokenize(text)
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Sentiment analysis

In [9]:
# Hugging Face sentiment analysis
tokenizer = XLNetTokenizer.from_pretrained('minh21/XLNet-Reddit-Sentiment-Analysis')
# Try to use minh21 repo but it doesn't work
model = XLNetModel.from_pretrained('xlnet/xlnet-base-cased')

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]